# PEFT - LoRA

https://huggingface.co/docs/peft/en/developer_guides/lora


**1. 다양한 초기화 전략 (Initialization)**

LoRA 가중치를 어떻게 시작하느냐에 따라 학습 속도와 성능이 달라진다.

* **기본값:** 가중치 A는 Kaiming-uniform, B는 0으로 초기화하여 처음에는 정등 변환(Identity transform) 상태로 시작한다.
* **PiSSA:** 주특이값(Principal singular values)을 사용하여 초기화하며, 일반 LoRA보다 수렴 속도가 빠르고 성능이 우수하다.
* **OLoRA:** QR 분해를 활용하여 베이스 모델의 가중치를 변환하며, 학습 안정성과 수렴 속도를 높인다.
* **EVA:** 입력 활성화 값에 대해 SVD를 수행하여 데이터 기반으로 초기화하며, 레이어별로 랭크(Rank)를 유연하게 할당한다.

**2. 양자화 모델 최적화 (LoftQ & DoRA)**

* **LoftQ:** 양자화된 모델을 미세 조정할 때 발생하는 오차를 최소화하도록 LoRA 가중치를 초기화하는 기술이다.
* **DoRA (Weight-Decomposed Low-Rank Adaptation):** 가중치 업데이트를 크기(Magnitude)와 방향(Direction)으로 분리하여 처리하며, 특히 낮은 랭크에서도 높은 성능을 보여준다.

**3. 효율적인 추론 및 학습 기법**

* **aLoRA (Activated LoRA):** 특정 토큰(invocation tokens)이 나타날 때만 어댑터를 활성화하는 방식이다. 기본 모델과 KV 캐시를 공유할 수 있어 추론 속도를 획기적으로 높일 수 있다.
* **Rank-stabilized LoRA (rsLoRA):** 랭크()의 제곱근에 비례하여 스케일링을 조절함으로써, 높은 랭크를 사용할 때 학습을 안정화시킨다.
* **레이어 복제 (Layer Replication):** 기존 모델의 레이어를 논리적으로 복제하여 모델 크기를 확장하되, 실제 메모리 사용량은 최소화하면서 어댑터를 추가하는 방식이다.

**4. 고급 최적화 및 제어**

* **특수 옵티마이저:** 가중치 A를 고정하고 B만 튜닝하여 메모리를 아끼는 **LoRA-FA**, A와 B에 서로 다른 학습률을 적용해 속도를 2배 높이는 **LoRA+** 등을 지원한다.
* **세부 제어:** 특정 레이어마다 서로 다른 랭크()나 알파() 값을 지정할 수 있는 기능을 제공한다.
* **토큰 학습:** 특정 레이어의 가중치뿐만 아니라, 특정 토큰 임베딩만 선택적으로 학습시키는 기능을 지원한다.

---

In [1]:
%pip install -Uq transformers datasets accelerate trl peft pydantic langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [2]:
!nvidia-smi  # GPU 모델명 / 드라이브 버전 / 메모리.사용량 확인
# 맥은 zsh:1: command not found: nvidia-smi 이게 정상

zsh:1: command not found: nvidia-smi


In [3]:
# Local 실행인 경우 아래 코드로 키를 설정
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("openai_key")
HF_TOKEN = os.getenv("HF_TOKEN")

In [4]:
# # Runpod 실행인 경우 아래 코드로 키 설정
# import os

# load_dotenv()
# OPENAI_API_KEY = os.environ("OPENAI_API_KEY")
# HF_TOKEN = os.environ("HF_TOKEN")

## 데이터셋 로드
https://huggingface.co/datasets/jwoooooooo/naver-economy-news2stock

In [5]:
from datasets import load_dataset # HuggingFace 데이터셋 로더

# Hub에서 split이 train인 데이터로드
dataset = load_dataset('jwoooooooo/naver-economy-news2stock', split='train')

print(len(dataset))
print(dataset)

/opt/anaconda3/envs/nlp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1000
Dataset({
    features: ['system', 'user', 'assistant'],
    num_rows: 1000
})


In [6]:
dataset[100] # 101번째 샘플을 확인 (딕셔너리)

{'system': "\n당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고, \n특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.\n\n다음 출력지시사항을 지켜주세요.\n1. 뉴스와 종목간의 연관성을 발견할 수 없다면:\n    - stock_related를 False로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n2. 뉴스와 종목간의 연관성을 발견했다면:\n    - stock_related를 True로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n    - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.\n    - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.\n    - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.\n",
 'user': '공유수면 사용하려면 어입인 의견 들어야\n해수부 5일 개정 공유수면 관리 및 매립에 관한 법률 시행 헤럴드경제 홍태화 기자 앞으로 공유수면관리청이 어업·환경 등에 영향을 미칠 것으로 예상되는 공유수면 점용·사용 허가를 할 때 미리 어업인 등 이해관계자들의 의견을 들어야 한다. 해양수산부는 5일 이같은 내용이 담긴 개정 공유수면 관리 및 매립에 관한 법률과 같은 법 시행령·시행규칙이 이날부터 시행된다고 밝혔다. 바다·바닷가·하천 등 공유수면은 공유재이기 때문에 이를 점용·사용하기 위해서는 별도의 허가를 받아야 한다. 최근 해상풍력 발전시설 해변을 이용한 관광시설 등 대규모 시설이 공유수면을 장기적으로 점용·사용하는 경우가 늘어났지만 이해 관계자의 의견을 사전에 수렴할 수 없는 문제가 있었다. 이에 공유수면 점용·사용으로 인한 사회적 갈등이 증가했다. 이러한 문제를 해결하기 위해 해수부는 지난 1월 공유수면

# 데이터셋 분할

In [7]:
test_ratio = 0.2    # 평가셋 비율

train_data = []     # 학습 데이터 리스트
test_data = []      # 테스트 데이터 리스트

data_indices = list(range(len(dataset)))        # 전체 인덱스
test_size = int(len(dataset) * test_ratio )     # 테스트 셋 크기

test_data_indices = data_indices[:test_size]    # 앞부분은 평가셋으로 사용
train_data_indices = data_indices[test_size:]   # 나멎머지는 학습셋으로 사용

# 학습/평가 데이터셋 형식을 지정하는 함수
def format_data(data):
    # OpenAI / Chat 학습용 messages 포멧 설정
    return {
        'messages' : [
            {
                'role' :'system',
                'content' : data['system']
            },
            {
                'role' :'user',
                'content' : data['user']
            },
            {
                'role' :'assistant',
                'content' : data['assistant']
            },
        ]
    }

# 학습 / 평가 인덱스를 변환해 리스트 생성
train_data = [format_data(dataset[i]) for i in train_data_indices]
test_data = [format_data(dataset[i]) for i in test_data_indices]

print(len(train_data))
print(len(test_data))

800
200


In [8]:
print(train_data[100])     # 학습 데이터의 101번째 샘플 확인

print(type(train_data))     # train_data는 list

{'messages': [{'role': 'system', 'content': "\n당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고, \n특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.\n\n다음 출력지시사항을 지켜주세요.\n1. 뉴스와 종목간의 연관성을 발견할 수 없다면:\n    - stock_related를 False로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n2. 뉴스와 종목간의 연관성을 발견했다면:\n    - stock_related를 True로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n    - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.\n    - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.\n    - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.\n"}, {'role': 'user', 'content': '송옥렬 공정위원장 후보자 첫 출근길\n서울 연합뉴스 임화영 기자 송옥렬 공정거래위원장 후보자가 5일 오전 서울 중구 한국공정거래조정원에 마련된 인사청문회 사무실로 첫 출근을 하며 취재진의 질문에 답하고 있다.'}, {'role': 'assistant', 'content': '{"stock_related":false,"summary":"송옥렬 공정거래위원장 후보자가 인사청문회 사무실로 첫 출근했다는 소식으로, 인사와 관련된 기본적인 출근 동정 뉴스다.","positive_stocks":[],"positive_keywords":[],"positive_reasons":"","negative_stocks":[],"negative_keywords":[],"negative_reasons":""}'}]}
<class 'li

In [9]:
from datasets import Dataset    # Huggingface Dataset 컨테이너

train_dataset = Dataset.from_list(train_data)  # train_data(list) -> Dataset 변환
test_dataset = Dataset.from_list(test_data)     #  test_data(list) -> Dataset 변환


print(train_dataset[100])
print(test_dataset[100])    # train_dataset은 Dataset형

{'messages': [{'content': "\n당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고, \n특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.\n\n다음 출력지시사항을 지켜주세요.\n1. 뉴스와 종목간의 연관성을 발견할 수 없다면:\n    - stock_related를 False로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n2. 뉴스와 종목간의 연관성을 발견했다면:\n    - stock_related를 True로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n    - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.\n    - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.\n    - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.\n", 'role': 'system'}, {'content': '송옥렬 공정위원장 후보자 첫 출근길\n서울 연합뉴스 임화영 기자 송옥렬 공정거래위원장 후보자가 5일 오전 서울 중구 한국공정거래조정원에 마련된 인사청문회 사무실로 첫 출근을 하며 취재진의 질문에 답하고 있다.', 'role': 'user'}, {'content': '{"stock_related":false,"summary":"송옥렬 공정거래위원장 후보자가 인사청문회 사무실로 첫 출근했다는 소식으로, 인사와 관련된 기본적인 출근 동정 뉴스다.","positive_stocks":[],"positive_keywords":[],"positive_reasons":"","negative_stocks":[],"negative_keywords":[],"negative_reasons":""}', 'role': 'assistant'}]}
{'messages

---

## NCSOFT/Llama-VARCO-8B-Instruct란?
https://huggingface.co/NCSOFT/Llama-VARCO-8B-Instruct


* **기반 모델:** Meta의 Llama-3.1-8B 모델을 기반으로 한다.
* **개발 목적:** 한국어 능력을 극대화하는 동시에 영어 구사 능력도 유지하도록 설계되었다.
* **학습 방법:** 한국어와 영어 데이터셋을 활용한 지속 사전 학습(Continual Pre-training)을 거쳤으며, 이후 지도 미세 조정(SFT)과 직접 선호도 최적화(DPO)를 통해 인간의 선호도에 맞게 정렬되었다.


**SFT에서 한국어능력향상과 동시에 영어능력유지란:**

일반적으로 한국어 데이터를 대량으로 추가 학습시키면 기존에 모델이 가지고 있던 영어 지식이 손상되는 '파괴적 망각(Catastrophic Forgetting)' 현상이 발생한다. 엔씨소프트는 이를 방지하기 위해 **지속 사전 학습(Continual Pre-training)**을 적용했다.

**_1. 데이터 믹스(Data Mixing) 전략:_**

단순히 한국어 데이터만 밀어 넣는 것이 아니라, 모델이 이미 학습했던 영어 데이터와 고품질의 한국어 데이터를 특정 비율로 섞어 학습한다. 이를 통해 기존의 영어 추론 능력을 '복습'하면서 새로운 언어 체계를 '습득'하게 된다.

**_2. 토크나이저 효율화와 임베딩 확장:_**

기존 Llama-3.1의 토크나이저 성능을 유지하면서 한국어 표현력을 높이기 위해 어휘 사전(Vocabulary)을 최적화한다. 영어 토큰 정보는 건드리지 않고 한국어 토큰의 밀도를 높여 두 언어 간의 연결 고리를 강화하는 방식이다.

**_3. 지식 전이(Knowledge Transfer):_**

영어 데이터로 학습된 모델의 강력한 논리적 사고 능력을 한국어로 전이시키는 과정을 거친다.

* **추론 능력 유지:** 수학이나 코딩 같은 논리적 작업은 영어 데이터에서 배운 구조를 그대로 활용한다.
* **언어 정렬:** SFT(지도 미세 조정) 단계에서 동일한 질문을 한국어와 영어로 번급하며 학습시켜, 언어에 상관없이 일관된 답변을 내놓도록 유도한다.

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM    # 토크나이저 / 생성형 모델 로더
import torch

pretrained_model_name = 'NCSOFT/Llama-VARCO-8B-Instruct'    # 사용할 사전학습 모델 ID

# 사전학습 CauselLM 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name,     # 모델 이름
    dtype = torch.bfloat16,    # 가중치 로딩 dtype(bf16)
    device_map = 'auto'         # 환경에 맞게 CPU/GPU 자동 배치
)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)     # 해당 모델의 토크나이저 로드

Loading weights: 100%|██████████| 291/291 [00:07<00:00, 39.19it/s, Materializing param=model.norm.weight]                               
Some parameters are on the meta device because they were offloaded to the disk.


## llama-3 chat template 변환

Llama3 모델은 특정 chat template 형식으로 학습되어, 그 형식을 사용해야 최적 성능을 낼 수 있다.
Chat template을 사용하지 않으면 모델이 대화 구조를 제대로 인식하지 못할 수 있다.
opean_ai 형식의 데이터를 llama-3 형식으로 변환한다.


**LLaMA-3 채팅 포맷**
LLaMA-3 채팅 포맷은 LLaMA-3 계열 챗봇 모델이 대화 내용을 이해하고 답변할 수 있도록 만들어진 입력 데이터 구조입니다.
여러 역할(시스템, 유저, 어시스턴트)의 메시지를 특별한 토큰과 구조로 묶어서 하나의 프롬프트로 합치는 방식입니다.
구조 예시
아래와 같이 대화 흐름을 명확히 구분하는 토큰들이 사용됩니다:

```
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
[시스템 역할 지침]<|eot_id|>
<|start_header_id|>user<|end_header_id|>
[유저 질문]<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
[모델의 답변]<|eot_id|>
```
* <|begin_of_text|> : 전체 프롬프트의 시작을 알리는 토큰
* <|start_header_id|>role<|end_header_id|> : 각 메시지의 역할 구분(시스템, 유저, 어시스턴트 등)
* 각 메시지 끝에 <|eot_id|> : 하나의 메시지 블록이 끝났음을 알림
* 마지막 assistant 블럭은 응답 생성 위치를 가리킨다. apply_chat_template(add_generation_prompt=False)로 설정했더라도 내부 템플릿에는 응답을 받을 자리 표시자로 <|assistant|> 토큰이 남아 있어, "여기서부터 어시스턴트가 답변을 생성해야 한다"는 신호를 제공하는 것임.

**왜 이 포맷이 필요할까?**

* 모델이 **“어디까지가 시스템 안내, 어디서부터가 유저 질문, 어디서부터가 답변인지”** 정확하게 파악할 수 있다.
* 여러 턴(turn)의 대화가 이어질 때도 메시지 경계를 명확히 구분해 혼동 없이 맥락을 유지할 수 있다.
* LLaMA-3 계열 모델은 이런 포맷으로 학습되어 있기 때문에 **실전 파인튜닝/추론 시에도 반드시 이 구조로 입력해야** 기대하는 챗봇 성능을 발휘할 수 있다.

In [11]:
# apply_chat_template 함수
# - openai 방식의 메세지를 llama3 방식으로 변환
text = tokenizer.apply_chat_template(train_dataset[100]['messages'], tokenize=False)  # messages -> 채팅 프롬프트 문자열 변환
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고, 
특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.

다음 출력지시사항을 지켜주세요.
1. 뉴스와 종목간의 연관성을 발견할 수 없다면:
    - stock_related를 False로 작성하세요.
    - summary에 뉴스의 요약을 작성하세요.
2. 뉴스와 종목간의 연관성을 발견했다면:
    - stock_related를 True로 작성하세요.
    - summary에 뉴스의 요약을 작성하세요.
    - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.
    - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.
    - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

송옥렬 공정위원장 후보자 첫 출근길
서울 연합뉴스 임화영 기자 송옥렬 공정거래위원장 후보자가 5일 오전 서울 중구 한국공정거래조정원에 마련된 인사청문회 사무실로 첫 출근을 하며 취재진의 질문에 답하고 있다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{"stock_related":false,"summary":"송옥렬 공정거래위원장 후보자가 인사청문회 사무실로 첫 출근했다는 소식으로, 인사와 관련된 기본적인 출근 동정 뉴스다.","positive_stocks":[],"positive_keywords":[],"positive_reasons":"","negative_stocks":[],"negative_keywords":[],

### data_collator 함수

* 미니배치(batch) 데이터를 모델이 바로 학습할 수 있는 형태(토큰·마스크·정답)로 변환합니다.
* 특히 아래와 같은 LLaMA-3 채팅 포맷을 쓸 때,
  “어디까지가 질문/어디서부터가 답변(assistant)인지”를 정확히 구분해서
  모델이 정답(답변 부분)만 학습하도록 레이블을 지정합니다.

#### 함수 설명

**1. 프롬프트 생성 (Prompt Construction)**

입력받은 `batch` 데이터는 리스트 내에 여러 메시지(`system`, `user`, `assistant`)를 포함하는 사전(dict) 구조이다.

* Llama 3의 특수 토큰(` <|begin_of_text|>`, `<|start_header_id|>`, `<|eot_id|>`)을 사용하여 모든 대화 내용을 하나의 긴 문자열로 병합한다.
* 각 역할(role)의 시작과 끝을 명확히 구분하여 모델이 대화 맥락을 이해할 수 있도록 구성한다.

**2. 토크나이즈 및 패딩 (Tokenization)**

병합된 문자열 리스트를 `tokenizer`를 통해 숫자 ID(`input_ids`)로 변환한다.

* `padding=True`: 배치 내의 문장들 중 가장 긴 문장을 기준으로 길이를 맞춘다.
* `truncation=True`: `max_length`를 초과하는 데이터는 절단한다.
* `return_tensors="pt"`: PyTorch 텐서 형식으로 결과를 반환한다.

**3. 레이블 생성 및 Loss Masking**

이 함수의 핵심 부분이다. 모델이 '사용자의 질문'이 아닌 **'모델의 답변(assistant)'** 부분에 대해서만 학습하도록 설정한다.

* **-100 값의 의미**: PyTorch의 `CrossEntropyLoss`는 레이블 값이 `-100`인 경우 손실(Loss) 계산에서 제외한다. 이를 통해 모델은 질문 부분을 예측하려고 노력하지 않고, 답변 부분의 정확도에만 집중하게 된다.
* **구간 탐색**: `assistant_tokens`를 기점으로 답변이 시작되는 위치를 찾고, `<|eot_id|>` 토큰이 나오는 지점까지의 인덱스를 추출한다.
* **값 복사**: 해당 구간의 `labels`에만 실제 `input_ids` 값을 복사하여 넣는다.|

In [12]:
# chat 모델 학습용 데이터 콜레이터 : 프롬프트 생성 -> 토크나이저/패딩 -> assistant 구간만 라벨링

# 배치(messages)를 모델 학습 텐서로 변환하는 함수
def data_collator(batch, tokenizer=tokenizer, max_length = 8192):
    # 1.프롬프트 생성
    prompts = []   # 배치프롬프트 문자열을 담을 리스트
    for example in batch :
        prompt = '<|begin_of_text|>'   # 프롬프트 시작
        for msg in example['messages']:   # system/user/assistant 순회
            role = msg['role'] 
            content = msg['content'].strip()
            # role + content를 템플릿으로 누적
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"
        prompts.append(prompt)    # 완성된 프롬프트를 배치 리스트에 추가

    # 2. 토큰처리/패딩/텐서변환
    tokenized = tokenizer(        # 프롬프트를 토큰화해서 텐서로 변환
        prompts,                  # 배치 프롬프트 목록
        truncation = True,        # 최대길이 초과시 자름
        max_length = max_length,  # 최대 토큰 길이
        padding = True,           # 배치 내 최장 길이 기준 패딩
        return_tensors = 'pt'     # Pytorh 텐서 변환
    )
    input_ids = tokenized['input_ids']   # 토큰 id 텐서
    attention_mask = tokenized['attention_mask']   # 패딩 마스크 텐서
    

    # 3. 라벨 생성
    labels = torch.full_like(input_ids, fill_value = -100)    # 기본은 -100 (손실 계산에서 제외)
    assistant_header = '<|start_header_id|>assistant<|end_header_id|>'
    assistant_token_id = tokenizer.encode(assistant_header, add_special_tokens = False)     # 헤더의 토큰 패턴
    eot_token = '<|eot_id|>'
    eot_token_id = tokenizer.encode(eot_token, add_special_tokens = False)   # 종료 토큰의 토큰 패턴

    for i, ids in enumerate(input_ids):     # 배치 매 각 샘플별로 라벨 구간 설정
        ids_list = ids.tolist()
        # assistant 답변 시작위치 찾기
        # - <|start_header_id|>assistant<|end_header_id|>\n 다음 인덱스부터 답변 시작
        start = None 
        for idx in range(len(ids_list) - len(assistant_token_id) + 1):   # 헤더 길이만큼 슬라이딩 탐색
            if ids_list[idx: idx+ len(assistant_token_id)] == assistant_token_id:    # 헤더 패턴 매칭
                start = idx + len(assistant_token_id)     # 헤더 다음 토큰부터 라벨 시작
                break    # 첫번째 assistant 구간만 탐색
        # 답변 끝 위치 찾기
        # - <|eot_id|> 까지
        if start is not None:   # assistant 헤더를 찾은 경우
            end = None          # 답변 종료 인덱스 초기화
            # start 이후부터  <|eot_id|> 패턴이 나오는 곳까지를 탐색
            for idx in range(start, len(ids_list) - len(eot_token_id) + 1):
                if ids_list[idx: idx+ len(eot_token_id)] == eot_token_id:   # 종료 패턴 매칭
                    end = idx + len(eot_token_id)   # eot까지 포함한 구간 설정
                    break    # 첫 번째 eot 탐색 완료시 종료
        labels[i, start:end] = input_ids[i, start:end]   # assistant 답변 구간을 정답 라벨로 복사

    return {
        'input_ids': input_ids,                 # 모델 입력
        'attention_mask': attention_mask,       # 패딩 마스크
        'labels': labels                        # 손실 계산용 라벨 (assistant 구간)
    }

data_collator([train_dataset[0], train_dataset[1]])

{'input_ids': tensor([[128000, 128006,   9125,  ...,   1210,     92, 128009],
         [128000, 128006,   9125,  ...,      0,      0,      0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[  -100,   -100,   -100,  ...,   1210,     92, 128009],
         [  -100,   -100,   -100,  ...,   -100,   -100,   -100]])}

### Causal Language Model 파인튜닝: input_ids와 labels 구조 이해

**데이터 구조**
```
input_ids:  [system_tokens..., user_tokens..., assistant_tokens...]  # 전체 시퀀스
labels:     [-100, -100, ..., -100, assistant_tokens...]          # assistant만 학습 대상
```

| 항목 | 내용 |
| --- | --- |
| **Input IDs** | 프롬프트 + 정답 (전체 시퀀스) |
| **Labels** | `-100` (프롬프트 구간) + 정답 토큰 (답변 구간) |
| **결과** | 모델은 입력을 다 보지만, 오직 답변을 맞히는 과정에서만 학습이 일어남 |




> **질문에 해당하는 input_ids에 이미 답이 포함되어 있다!**
>
> **"답이 이미 있는데 어떻게 학습하는가?"**
>
> 모델은 정답을 "보면서" 각 위치에서 올바른 다음 토큰을 예측하는 법을 배운다. 마치 학생이 모범답안을 보며 "이 상황에서는 이렇게 답해야 한다"를 학습하는 것과 같다. 이것이 현대 LLM 파인튜닝의 핵심 메커니즘이다!


**_1. 인과적 언어 모델링 (Causal Language Modeling):_**

LLM(Llama, GPT 등)은 **이전 토큰들을 보고 다음 토큰을 예측**하는 방식으로 학습한다. 따라서 학습 데이터에는 프롬프트와 정답이 모두 포함된 전체 문장이 들어가야 한다.

* **학습 원리:** 모델은 번째 토큰까지를 입력으로 받아 번째 토큰을 예측한다.
* **구조:** `input_ids`가 `[A, B, C, D]`라면, 모델은 내부적으로 `A`를 보고 `B`를, `A, B`를 보고 `C`를 예측하는 과정을 동시에 수행한다.

**_2. Teacher Forcing 기법:_**
```
Position:   [0, 1, 2, 3, 4, 5, 6, 7, 8]
input_ids:  [A, B, C, D, E, F, G, H, I]
labels:     [-100, -100, -100, -100, E, F, G, H, I]
```

학습 과정:
- Position 4: A,B,C,D를 보고 → E 예측
- Position 5: A,B,C,D,E를 보고 → F 예측  
- Position 6: A,B,C,D,E,F를 보고 → G 예측

**_3. Labels와 Loss 계산의 역할:_**

`input_ids`에 정답이 포함되어 있더라도, 모델이 모든 구간에 대해 학습(손실 계산)을 수행하는 것은 아니다. 이때 중요한 역할을 하는 것이 바로 코드에 작성된 **`labels`**이다.

* **-100의 의미:** PyTorch의 `CrossEntropyLoss`는 기본적으로 레이블 값이 `-100`인 위치를 무시(ignore)한다.
* **학습 차단:** 코드에서 프롬프트(User 질문 등) 구간의 레이블을 `-100`으로 설정했기 때문에, 모델이 프롬프트 내용을 예측하며 발생하는 오차는 학습에 반영되지 않는다.
* **학습 집중:** 오직 `assistant`의 답변 구간에 해당하는 `labels`만 실제 `input_ids` 값을 가지므로, 모델은 **"프롬프트가 주어졌을 때 정답을 생성하는 방법"**에 대해서만 가중치를 업데이트한다.


**학습 vs 추론의 차이**

**_학습 시:_**
```
input_ids: <system>당신은 금융분석가</system><user>뉴스내용</user><assistant>분석결과</assistant>
labels:    [-100, -100, ..., -100, 분석결과_토큰들]
```

**_추론 시:_**
```
input:  <system>당신은 금융분석가</system><user>뉴스내용</user><assistant>
output: 분석결과 (모델이 한 토큰씩 생성)
```

In [17]:
example = train_dataset[128]                # 129번째 샘플
batch = data_collator([example])            # 배치 1개로 콜레이터 적용

# print(f'{batch['input_ids'].shape}')        # input_ids 텐서 shape 확인
# print(f'{batch['attention_mask'].shape}')   # attention_mask 텐서 shape 확인
# print(f'{batch['labels'].shape}')           # labels 텐서 shape 확인

In [18]:
print(batch['input_ids'][0].tolist())           # 샘플의 input_ids를 리스트로 확인
print(batch['attention_mask'][0].tolist())      # 샘플의 attention_mask를 리스트로 확인
print(batch['labels'][0].tolist())              # 샘플의 labels를 리스트로 확인

[128000, 128006, 9125, 128007, 198, 65895, 83628, 34804, 104193, 123061, 14, 127463, 111068, 120226, 125959, 102612, 96318, 27797, 18359, 87097, 103168, 34983, 114942, 101360, 11, 720, 108159, 30381, 59134, 41953, 99458, 88708, 19954, 101412, 116129, 41871, 235, 30381, 14, 64189, 30381, 115754, 58126, 64189, 11, 111436, 11, 106589, 93292, 18918, 109862, 44005, 104193, 123061, 14, 127463, 109862, 116425, 20565, 80052, 382, 13447, 49531, 62226, 22035, 30426, 115790, 18359, 67890, 115061, 92769, 627, 16, 13, 111068, 25941, 81673, 99458, 88708, 63375, 21028, 78453, 101106, 111490, 121712, 48936, 29833, 47782, 115300, 512, 262, 482, 5708, 54356, 18918, 3641, 17835, 114839, 92245, 627, 262, 482, 12399, 19954, 111068, 120226, 87097, 103168, 18359, 114839, 92245, 627, 17, 13, 111068, 25941, 81673, 99458, 88708, 63375, 21028, 78453, 101106, 111490, 121712, 101528, 33390, 512, 262, 482, 5708, 54356, 18918, 3082, 17835, 114839, 92245, 627, 262, 482, 12399, 19954, 111068, 120226, 87097, 103168, 18

In [19]:
label_ids = [token_id for token_id in batch['labels'][0].tolist() if token_id != -100]  # 정답 토큰 (-100 제외)만 추출
text = tokenizer.decode(label_ids)   # 정답 토큰을 문자열로 디코딩
text

'\n{"stock_related":true,"summary":"에어부산이 코로나19 팬데믹 이후 28개월 만에 김해국제공항에서 몽골 울란바토르와 일본 오사카 노선 운항을 주 2회 재개하며, 곧 코타키나발루, 나트랑, 세부 등 동남아 노선도 운항을 시작한다. 인천공항발 국제 노선 확대도 예고했다. 몽골의 무격리·무백신 입국 정책, 일본 오사카 노선 등 해외여행 수요 증대를 선제적으로 겨냥한 조치이다.","positive_stocks":["에어부산"],"positive_keywords":["국제선 운항 재개","해외여행 수요 확대","동남아·일본 노선 확대","코로나 이후 노선 복원"],"positive_reasons":"팬데믹 이후 억눌렸던 해외여행 수요 급증 기대, 국제선 노선 재개 및 증편으로 에어부산의 항공 노선 수익 확대가 예상되어 긍정적 영향을 줄 수 있음. 특히 일본, 동남아, 몽골 등 인기 지역 노선의 조기 복원은 매출 증가로 직결될 가능성이 높음.","negative_stocks":[],"negative_keywords":[],"negative_reasons":""}<|eot_id|>'

In [20]:
tokens = tokenizer.convert_ids_to_tokens(batch['input_ids'][0].tolist())  # 토큰 ID를 토큰 문자열로 변환

text_tokens = []    # 토큰 ID를 1개씩 디코딩한 문자열을 담을 리스트
for i, token_id in enumerate(batch['input_ids'][0].tolist()):    # 토큰 ID를 순회
    decoded_str = tokenizer.decode([token_id])      # 토큰 1개를 문자열로 디코딩
    text_tokens.append(decoded_str)                 # 디코딩 결과를 누적 저장

print(text_tokens)

['<|begin_of_text|>', '<|start_header_id|>', 'system', '<|end_header_id|>', '\n', '당', '신', '은', ' 금', '융', '/', '경제', ' 뉴', '스의', ' 핵', '심', '내', '용', '을', ' 요', '약', '해', ' 설명', '하고', ',', ' \n', '특', '정', ' 상', '장', ' 종', '목', '에', ' 미', '치는', ' �', '�', '정', '/', '부', '정', ' 영향', '여', '부', ',', ' 이유', ',', ' 근', '거', '를', ' 분석', '하는', ' 금', '융', '/', '경제', ' 분석', ' 전문', '가', '입니다', '.\n\n', '다', '음', ' 출력', '지', '시', '사항', '을', ' 지', '켜', '주세요', '.\n', '1', '.', ' 뉴', '스', '와', ' 종', '목', '간', '의', ' 연', '관', '성을', ' 발견', '할', ' 수', ' 없', '다면', ':\n', '   ', ' -', ' stock', '_related', '를', ' False', '로', ' 작성', '하세요', '.\n', '   ', ' -', ' summary', '에', ' 뉴', '스의', ' 요', '약', '을', ' 작성', '하세요', '.\n', '2', '.', ' 뉴', '스', '와', ' 종', '목', '간', '의', ' 연', '관', '성을', ' 발견', '했다', '면', ':\n', '   ', ' -', ' stock', '_related', '를', ' True', '로', ' 작성', '하세요', '.\n', '   ', ' -', ' summary', '에', ' 뉴', '스의', ' 요', '약', '을', ' 작성', '하세요', '.\n', '   ', ' -', ' �', '�', '정', '영', '향', '

In [21]:
import pandas as pd

df = pd.DataFrame({
    'token' : text_tokens,      # 토큰 (1개씩 디코딩된 문자열)
    'input_ids' : batch['input_ids'][0].tolist(),    # 입력 토큰 ID
    'attention_mask' : batch['attention_mask'][0].tolist(),   # 패딩 여부 (1/0)
    'labels' : batch['labels'][0].tolist()      # 정답 라벨 (-100 마스킹 포함)
}).transpose()   # 컬럼을 행으로 전치

pd.set_option('display.max_columns', None)       # 출력시 컬럼 생략 없이 표시
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915
token,<|begin_of_text|>,<|start_header_id|>,system,<|end_header_id|>,\n,당,신,은,금,융,/,경제,뉴,스의,핵,심,내,용,을,요,약,해,설명,하고,",",\n,특,정,상,장,종,목,에,미,치는,�,�,정,/,부,정,영향,여,부,",",이유,",",근,거,를,분석,하는,금,융,/,경제,분석,전문,가,입니다,.\n\n,다,음,출력,지,시,사항,을,지,켜,주세요,.\n,1,.,뉴,스,와,종,목,간,의,연,관,성을,발견,할,수,없,다면,:\n,,-,stock,_related,를,False,로,작성,하세요,.\n,,-,summary,에,뉴,스의,요,약,을,작성,하세요,.\n,2,.,뉴,스,와,종,목,간,의,연,관,성을,발견,했다,면,:\n,,-,stock,_related,를,True,로,작성,하세요,.\n,,-,summary,에,뉴,스의,

In [22]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r = 8,
    lora_alpha = 32,
    lora_dropout = 0.1,
    bias = 'none',
    target_modules = ['q_proj', 'v_proj'],
    task_type = 'CAUSAL_LM'
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


In [23]:
# SFTConfig
from trl import SFTConfig  # TRL SFT 학습 설정 클래스

hub_model_id = 'jwoooooooo/Llama-VARCO-8b-news2stock-analyzer'  # 학습 완료 후 업로드할 Hub 모델 ID

sft_config = SFTConfig(  # SFT 학습 하이퍼파라미터/저장/로그 설정
    output_dir="Llama-VARCO-8b-news2stock-analyzer", # 학습 완료된 모델과 체크포인트가 저장될 경로이다.
    num_train_epochs=3,                              # 전체 데이터셋을 반복 학습할 횟수(Epoch)이다.
    per_device_train_batch_size=2,                   # 각 GPU(장치)당 한 번에 처리할 데이터 샘플의 개수이다.
    gradient_accumulation_steps=2,                   # 그래디언트를 2번 누적한 후 가중치를 업데이트한다. (실제 배치 크기 = 2 * 2 = 4 효과를 낸다.)
    gradient_checkpointing=True,                     # VRAM 절약을 위해 중간 활성화 값을 저장하지 않고 역전파 시 재계산하는 설정이다.
    optim="adamw_torch_fused",                       # 최적화 알고리즘 설정이다. fused 버전은 CUDA에서 더 빠르다.
    logging_steps=10,                                # 10 스텝마다 학습 로그(Loss 등)를 출력한다.
    save_strategy="steps",                           # 체크포인트 저장 기준을 'steps'(스텝 수)로 설정한다. (옵션: 'epoch')
    save_steps=50,                                   # 50 스텝마다 모델 체크포인트를 저장한다.
    bf16=True,                                       # BF16(Brain Float 16) 정밀도를 사용하여 메모리를 아끼고 연산 속도를 높인다. (Ampere GPU 이상 권장)
    learning_rate=1e-4,                              # 학습률(Learning Rate)이다. 가중치 업데이트의 크기를 결정한다.
    max_grad_norm=0.3,                               # 그래디언트 클리핑 임계값이다. 그래디언트 폭주를 막아 학습 안정성을 높인다.
    warmup_ratio=0.03,                               # 전체 학습 단계의 3% 동안 학습률을 서서히 올리는 웜업(Warmup)을 수행한다.
    lr_scheduler_type="constant",                    # 학습률 스케줄러 타입이다. 여기서는 학습률을 변동 없이 상수로 유지한다.
    push_to_hub=True,                                # 학습이 끝나면 Hugging Face Hub에 모델을 자동으로 업로드한다.
    hub_model_id=hub_model_id,                       # Hub에 업로드될 때 사용될 저장소(Repository) ID이다.
    hub_token=True,                                  # Hub 업로드를 위해 인증 토큰을 사용한다.
    remove_unused_columns=False,                     # 데이터셋에서 모델의 forward 메서드 시그니처에 없는 컬럼을 자동으로 삭제하지 않도록 한다.
    dataset_kwargs={"skip_prepare_dataset": True},   # 데이터셋 처리 과정(packing 등)을 건너뛰도록 하는 설정이다.
    report_to=[],                                    # 학습 기록을 전송할 툴(WandB, Tensorboard 등)을 지정한다. 빈 리스트는 기록하지 않음을 의미한다.
    label_names=["labels"],                          # 손실(Loss) 계산 시 정답(Target)으로 사용할 데이터셋의 컬럼 이름이다.
)

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


In [25]:
from trl import SFTTrainer  # SFT 학습용 Trainer

trainer = SFTTrainer(
    model = model,
    args = sft_config,
    train_dataset = train_data,     # 학ㄱ습 데이터셋
    data_collator = data_collator   # 배치 텐서 생성 함수
)

trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
/opt/anaconda3/envs/nlp_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


RuntimeError: Function MmBackward0 returned an invalid gradient at index 1 - expected device meta but got mps:0

In [26]:
# 테스트셋 messages에서 프롬프트/정답(assistant) 텍스트 분리
prompt_list = []        # 프롬프트(assistant 메세지 이전) 저장 리스트
label_list = []         # 정답 (assistant 답변) 저장 리스트

for message in test_dataset['messages']:
    text = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False)  # 채팅 템플릿 문자열로 변환
    # assistant 시작 전까지는 프롬프트로 사용 + assistant 헤더만 프롬프트 끝에 붙여준다.
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + \
        '<|start_header_id|>assistant<|end_header_id|>\n'  

    # assistant 답변 (종료 토큰 전)만 추출
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]

    prompt_list.append(input)
    label_list.append(label)

In [27]:
prompt_list[100]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고, \n특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.\n\n다음 출력지시사항을 지켜주세요.\n1. 뉴스와 종목간의 연관성을 발견할 수 없다면:\n    - stock_related를 False로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n2. 뉴스와 종목간의 연관성을 발견했다면:\n    - stock_related를 True로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n    - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.\n    - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.\n    - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n공유수면 사용하려면 어입인 의견 들어야\n해수부 5일 개정 공유수면 관리 및 매립에 관한 법률 시행 헤럴드경제 홍태화 기자 앞으로 공유수면관리청이 어업·환경 등에 영향을 미칠 것으로 예상되는 공유수면 점용·사용 허가를 할 때 미리 어업인 등 이해관계자들의 의견을 들어야 한다. 해양수산부는 5일 이같은 내용이 담긴 개정 공유수면 관리 및 매립에 관한 법률과 같은 법 시행령·시행규칙이 이날부터 시행된다고 밝혔다. 바다·바닷가·하천 등 공유수면은 공유재이기 때문에 이를 점용·사용하기 위해서는 별도의 허가를 받아야 한다. 최근 해상풍력 발전시설 해변을 이용한 관광시설 등 대규모 시설이 공유수면을 장기적으로 점용·사용하는 경우가 늘어났지만 이해 관계자의 의

In [28]:
label_list[100]

'\n{"stock_related":true,"summary":"해양수산부가 공유수면 관리 및 매립에 관한 법률을 개정해, 해상풍력·관광시설 등 공유수면(바다·강 등) 점용·사용 허가 시 어업인 등 이해관계자 의견을 반드시 수렴하도록 의무화했다. 이는 공유수면 사용으로 인한 사회적 갈등 및 어업인 피해를 방지하기 위한 조치로, 앞으로 대규모 해상 개발사업 진행 과정에서 이해관계자의 이견 수렴과정이 강화된다.","positive_stocks":[],"positive_keywords":[],"positive_reasons":"","negative_stocks":["코오롱글로벌","씨에스윈드","유니슨","두산에너빌리티"],"negative_keywords":["해상풍력","공유수면 허가","어업인 의견청취","사회적 갈등 가능성","사업 절차 복잡화"],"negative_reasons":"공유수면 점용/사용 허가에 어업인 등 이해관계자의 의견 수렴 절차가 추가되어, 해상풍력 및 연관 해양개발 사업의 인허가 절차가 길어지고 불확실성이 커질 수 있다. 이에 따라 해상풍력발전소 구축, 해상개발, 관련 EPC·터빈업체에 단기적으로는 부정적인 영향이 예상된다."}'

### 추론모델 - 런타임결합
1. lora모델을 로드
2. base모델 + lora adapter 세팅

In [29]:
from peft import AutoPeftModelForCausalLM       # Hub에 저장된PEFT 모델 로더
from transformers import AutoTokenizer, pipeline    # 토크나이저 / 파이프라인 생성
import torch

peft_model_id = hub_model_id

finetuned_model = AutoPeftModelForCausalLM.from_pretrained(     # 파인튜닝 된 PEFT 모델 로드
    peft_model,
    torch_dtype=torch.float16,      # bf16 로드
    device_map="cpu",               # CPU에 로드
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)   # 같은 repo

pipe = pipeline("text-generation", model=finetuned_model, tokenizer=tokenizer)

pipe

NameError: name 'peft_model' is not defined

In [ ]:
eos_token = tokenizer('<|eot_id|>', add_special_tokens=False)['input_ids'][0]  # <|eot_id|>의 토큰 ID 추출
eos_token

In [ ]:
# 프롬프트를 입력 받으면 assistant 생성 결과만 반환하는 함수
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id= eos_token, do_sample=False)  # 응답 생성
    assistant_start = len(prompt)       # 생성 결과에서 프롬프트 길이만큼은 입력 구간
    return outputs["generated_ids"][assistant_start:].split()   # 프롬프트 이후 (생성된 부분)만 잘라서 반환

for prompt, label in zip(prompt_list[10:13], label_list[10:13]):
    print(f'[prompt]\n{prompt}')
    print(f'[label]\n{label}')
    print(f'[response]\n{test_inference(pipe, prompt)}')
    print('='* 100)
    

# 추론

In [ ]:
from numpy import False_
# 뉴스 본문을 넣으면 모델의 분석 응답 텍스트를 반환하는 함수
def inference():
    messages = [
        {'role': 'system', 'content': '''
당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고,
특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.

다음 출력지시사항을 지켜주세요.
1. 뉴스와 종목간의 연관성을 발견할 수 없다면:
    - stock_related를 False로 작성하세요.
    - summary에 뉴스의 요약을 작성하세요.
2. 뉴스와 종목간의 연관성을 발견했다면:
    - stock_related를 True로 작성하세요.
    - summary에 뉴스의 요약을 작성하세요.
    - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.
    - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.
    - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.
'''},  # 시스템 지시문
        {'role': 'user', 'content': news}  # 사용자 입력(뉴스)
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    outputs = pipe(
        prompt,
        max_new_tokens = 1024,
        eos_token_id = eos_token,
        do_sample = False
    )
    assistant_start = len(prompt)

    return outputs[0]['generated_text'][assistant_start:].strip()

In [ ]:
news = '''
강훈식 청와대 비서실장이 이끄는 캐나다 방산 특사단 출장 기간 현지에서 한·캐나다 자동차 포럼이 열린다. 한국과 캐나다의 자동차 산업 협력에 대한 논의가 이뤄질 전망이다. 이자리에는 정의선 현대자동차그룹 회장 등 주요 경영진도 참석할 것으로 알려졌다.

26일 자동차 업계 및 정부 관계자에 따르면 정 회장과 장재훈 부회장은 이번주 캐나다에서 열리는 ‘한국·캐나다 자동차 산업 협력 포럼’에 참석한다. 이 행사에는 김정관 산업통상자원부 장관과 멜라니 졸리 캐나다 산업부 장관 등 주요 인사가 모여 자동차 산업 협력 방안에 대해 논의할 예정이다. 정 회장이 현지 일정상 참석이 어려울 경우 장재훈 현대차 부회장만 참석할 가능성도 남아있다.

정 회장은 현지에서 구체적인 캐나다 투자 방안을 공개할 예정이다. 현대차는 캐나다 자원 등 장점을 활용해 수소 분야를 포함한 다양한 협력 방안을 검토 중인 것으로 알려졌다. 캐나다 정부가 요구해 온 ‘전기차 전용 공장 건설’은 투자 명단에서 제외하기로 가닥이 잡혔다. 북미 시장 공략을 위해 지난해 초 미국 조지아주에 완공한 메타플랜트아메리카(HMGMA)와의 중복 투자를 피하기 위해서다.

정 회장은 60조원 규모의 캐나다 초계 잠수함 사업(CPSP) 수주를 지원하기 위해 캐나다 출장길에 올랐다. 정 회장이 전면에 나선 배경에는 ‘절충교역’이 있다. 절충교역은 대규모 방산 계약을 발주하는 국가가 수주국에 현지 투자나 기술이전, 공급망 구축 등을 요구하는 방식이다. 캐나다 정부는 3000t급 디젤 잠수함 12척을 도입하는 대가로 현대차의 현지 투자를 강력히 희망해 왔다. 캐나다는 한국과 경쟁 상대인 독일 측에도 폭스바겐의 현지 생산 확대를 입찰 조건으로 제시한 것으로 전해졌다. 현대차는 캐나다에 생산 시설이 없는 반면 독일 폭스바겐은 배터리 셀 공장을 건설 중이다.

이번 수주전의 성패는 향후 30년간의 유지·보수·정비(MRO) 시장 주도권과 직결된다. 이 사업은 디젤 잠수함 최대 12척을 건조하는 프로젝트다. 건조비용(약 20조원)에 도입 후 30년간 유지·보수·운영(MRO) 비용까지 포함하면 최대 60조원까지 규모가 커질 전망이다.
'''
inference(news)  # news 문자열을 입력으로 넣어 모델 분석 결과(assistant 응답)를 생성

In [ ]:
news = '''
달러당 원화값이 26일 전 거래일 대비 20원 가까이 급등하며 1440원대를 이어가고 있다.

이날 서울외환시장에서 원화값은 전 거래일보다 19.7원 오른 1446.1원에 출발해 오전 10시25분 현재 1446.2원에 거래되고 있다.

미국과 일본 외환당국의 시장 개입 가능성이 거론되면서 엔화가 급등한 점이 원화 강세로 이어졌다는 분석이 나온다. 일본은행은 최근 외환시장 개입에 앞서 주요 은행을 상대로 거래 상황을 점검하는 ‘레이트 체크’를 실시한 것으로 전해졌다. 미국 뉴욕 연방준비은행도 미 재무부 지시에 따라 레이트 체크에 나섰다는 보도가 나왔다.

다카이치 사나에 일본 총리는 “투기적이고 비정상적인 움직임에 필요한 모든 조처를 할 것”이라고 밝힌 바 있다. 이에 따라 지난주 달러당 160엔에 육박했던 달러당 엔화값은 155엔대 초반까지 상승했다. 현재 엔화값은 전 거래일 대비 0.50% 오른 155.04엔이다.

한편 이날 열리는 국민연금 기금운용위원회에서 환헤지 전략 등이 논의될 예정이어서 원화값에 어떤 영향을 미칠지도 주목된다.
'''
inference(news)  # news 문자열을 입력으로 넣어 모델 분석 결과(assistant 응답)를 생성

### base모델과 비교

In [ ]:
# LoRA 파인튜닝 전(Base) vs 파인튜닝 후(LoRA) 모델 답변 비교
from transformers import AutoModelForCausalLM, pipeline  # 모델 로드/파이프라인 생성

base_model_id = "NCSOFT/Llama-VARCO-8B-Instruct"  # 베이스(파인튜닝 전) 모델 ID
base_model = AutoModelForCausalLM.from_pretrained(  # 베이스 모델 로드
    base_model_id,  # 모델 ID
    dtype=torch.bfloat16,  # BF16 로드
    device_map="auto",  # CPU/GPU 자동 배치
)
base_pipe = pipeline("text-generation", model=base_model, tokenizer=tokenizer)  # 베이스 모델 파이프라인 생성

for idx, (prompt, label) in enumerate(zip(prompt_list[10:13], label_list[10:13])):  # 샘플 3개만 비교
    print(f"[샘플 {idx + 1}]")  # 샘플 번호 출력
    base_resp = test_inference(base_pipe, prompt)  # 베이스 모델 응답 생성
    lora_resp = test_inference(pipe, prompt)  # LoRA 모델 응답 생성
    print(f"  [Base - 파인튜닝 전]\n{base_resp}")  # 베이스 모델 응답 출력
    print(f"  [LoRA - 파인튜닝 후]\n{lora_resp}")  # LoRA 모델 응답 출력
    print(f"  [Label]\n{label}")  # 정답(레이블) 출력
    print("-" * 50)  # 구분선 출력

In [ ]:
# Huggingface에 자동으로 업로드 되지 않을 경우 수동으로 업로드
import os
from huggingface_hub import HfApi

repo_id = "jwoooooooo/naver-economy-news2stock"
local_dir = './Llama-VARCO-8b-news2stock-analyzer'

api = HfApi(token=os.environ["HF_TOKEN"])
api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)
api.upload_folder(
    folder_path=local_dir,
    repo_id=repo_id,
    repo_type="model",
    ignore_patterns= ['checkpoints-*', '**/checkpoints-*'],
    commit_message="Upload filed LoRA adapter (without checkpoints)"
)